In [1]:
import pandas as pd
import numpy as np
import psycopg2
import pandas.io.sql as sqlio
import requests as re
from IPython.display import JSON
import json

### Http request from world Weather API
###### 10 most popular airport, for each month and year

In [2]:
flights = pd.read_csv('../../../data/flights/flightFinal.csv')

In [3]:
flights.head()

,Unnamed: 0,fl_date,mkt_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin_city_name,dest_airport_id,dest_city_name,crs_dep_time,crs_arr_time,arr_delay,crs_elapsed_time,distance,day,month,year,month_delay,carrier_delay,carrier_perc_delay
0,0,2018-11-10,WN,N282WN,2015,12892,"Los Angeles, CA",10140,"Albuquerque, NM",1235,1535,6.0,120.0,677,10,11,2018,3.454152,4.566613,19.68
1,1,2018-11-10,WN,N411WN,3788,12892,"Los Angeles, CA",10140,"Albuquerque, NM",1840,2125,1.0,105.0,677,10,11,2018,3.454152,4.566613,19.68
2,2,2018-11-10,WN,N556WN,3301,12892,"Los Angeles, CA",10397,"Atlanta, GA",805,1515,-12.0,250.0,1947,10,11,2018,3.454152,4.566613,19.68
3,3,2018-11-10,WN,N8648A,3525,12892,"Los Angeles, CA",10397,"Atlanta, GA",1045,1805,-20.0,260.0,1947,10,11,2018,3.454152,4.566613,19.68
4,4,2018-11-10,WN,N8550Q,854,12892,"Los Angeles, CA",10423,"Austin, TX",1030,1535,-12.0,185.0,1242,10,11,2018,3.454152,4.566613,19.68


In [4]:
depcities = flights.origin_city_name.unique()

In [5]:
destcities = flights.dest_city_name.unique()

In [6]:
len(destcities)

302

In [7]:
for index,element in enumerate(destcities):
    if element in depcities:
        print(element)
        destcities = np.delete(destcities,index)

Atlanta, GA
Denver, CO
Houston, TX
Chicago, IL
Phoenix, AZ
San Francisco, CA
Los Angeles, CA
Seattle, WA
Dallas/Fort Worth, TX
Charlotte, NC


In [8]:
len(destcities)

292

In [9]:
depcities[7] = 'Dallas, TX'

In [10]:
for index, city in enumerate(destcities):
    if '/' in city:
        splits = city.split(sep='/')
        city = splits[0] + ',' + splits[-1].split(sep=',')[-1]
        destcities[index] = city

In [11]:
start = flights['fl_date'].min()

In [12]:
start

'2018-01-01'

In [13]:
end = flights['fl_date'].max()

In [14]:
end

'2019-12-31'

In [15]:
years = ['2018','2019']
months = ['01','02','03','04','05','06','07','08','09','10','11','12']

for city in depcities:
    for year in years:
        for index,month in enumerate(months):
            day = '01'
            city2 = city.replace(" ","")
            city3 = city2.replace(",", "")
            if month != '12':
                nextmonth = months[index+1]
            else:
                nextmonth = '12'
                day = '31'
            res = re.get('http://api.worldweatheronline.com/premium/v1/past-weather.ashx',
                         params={'q':city,'date':year+'-'+month+'-'+'01',
                                 'enddate':year+'-'+nextmonth+'-'+day,
                                 'format':'json','key':'ff724f4ddab844c992b153222210106'})
            jsonRes = res.json()
            out_file = open("../../../data/weather/"+year+month+city3+".json", "w") 
            json.dump(jsonRes, out_file) 

In [19]:
for city in  destcities:
    city2 = city.replace(" ","")
    city3 = city2.replace(",", "")
    res = re.get('http://api.worldweatheronline.com/premium/v1/past-weather.ashx',params={'q':city,'date':'2018-01-01','enddate':'2019-12-31','format':'json','key':'ff724f4ddab844c992b153222210106'})
    jsonRes = res.json()
    out_file = open("../../../data/weather/"+city3+".json", "w") 
    json.dump(jsonRes, out_file) 